<a href="https://colab.research.google.com/github/YuxinZhang1998/S3-SIF/blob/main/Geo_features_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import datetime

ee.Authenticate()
ee.Initialize(project='ee-yuxinzhanguv1')

def export_lonlat_time(
    start_date: str,
    end_date: str,
    step_days: int,
    pixel_size: int,
    export_asset_folder: str,
    aoi: ee.Geometry
):
    """
    Generate lon/lat grid and timestamp images every N days
    and export them as assets.

    Args:
        start_date (str): Start date in 'YYYY-MM-DD'.
        end_date (str): End date in 'YYYY-MM-DD'.
        step_days (int): Interval in days between images.
        pixel_size (int): Spatial resolution in meters.
        export_asset_folder (str): Asset folder to export results.
        aoi (ee.Geometry): Region of interest.
    """

    # FeatureCollection wrapper (not strictly needed but kept from your code)
    fc = ee.FeatureCollection([ee.Feature(aoi)])

    # Step 1: 获取经纬度图像
    lonlat = ee.Image.pixelLonLat().clip(aoi).reproject('EPSG:4326', None, pixel_size)
    lon = lonlat.select('longitude').rename('lon')
    lat = lonlat.select('latitude').rename('lat')

    # Step 2: 日期列表
    start = ee.Date(start_date)
    end = ee.Date(end_date)
    n_days = end.difference(start, 'day').getInfo()
    dates = [start.advance(i, 'day') for i in range(0, int(n_days), step_days)]

    # Step 3: 创建影像集合
    def make_image(date):
        date = ee.Date(date)
        timestamp_img = ee.Image.constant(date.millis()) \
                        .rename('timestamp') \
                        .toFloat() \
                        .reproject('EPSG:4326', None, pixel_size) \
                        .clip(aoi)
        return lon.addBands(lat).addBands(timestamp_img) \
                  .set({'system:time_start': date.millis()})

    images = ee.ImageCollection([make_image(d) for d in dates])

    # Step 4: 导出
    imageList = images.toList(images.size())
    n_images = images.size().getInfo()

    bands_to_export = ['lon', 'lat']

    for i in range(n_images):
        try:
            image = ee.Image(imageList.get(i))
            image = image.select(bands_to_export).toFloat()

            time_start = image.get('system:time_start')
            date_str = ee.Date(time_start).format('YYYYMMdd').getInfo()

            image_to_export = image.set('date', date_str)

            export_name = f'Geo_export_{date_str}'
            asset_path = f"{export_asset_folder}/{export_name}"

            print(f"Processing: {export_name}")

            exportar = ee.batch.Export.image.toAsset(
                image=image_to_export,
                description=export_name,
                assetId=asset_path,
                region=aoi,
                scale=pixel_size,
                maxPixels=1e13
            )
            exportar.start()
            print(f"Export task created for {export_name}")

        except Exception as e:
            print(f"Error processing image at index {i}: {e}")
            continue

    print("All export tasks created.")



*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [ ]:
# ============================
# Example usage:
# ============================

aoi_global = ee.Geometry.Rectangle([-10, 36, 4, 44], geodesic=False);

export_lonlat_time(
    start_date="2019-01-01",
    end_date="2019-12-31",
    step_days=8,
    pixel_size=300,
    export_asset_folder="projects/ee-yuxinzhanguv1/assets/Geo-8DAYS-peninsula2019-300-FeatureCollection",
    aoi=aoi_global
)


Processing: Geo_export_20190101
Export task created for Geo_export_20190101
Processing: Geo_export_20190109
Export task created for Geo_export_20190109
Processing: Geo_export_20190117
Export task created for Geo_export_20190117
Processing: Geo_export_20190125
Export task created for Geo_export_20190125
Processing: Geo_export_20190202
Export task created for Geo_export_20190202
Processing: Geo_export_20190210
Export task created for Geo_export_20190210
Processing: Geo_export_20190218
Export task created for Geo_export_20190218
Processing: Geo_export_20190226
Export task created for Geo_export_20190226
Processing: Geo_export_20190306
Export task created for Geo_export_20190306
Processing: Geo_export_20190314
Export task created for Geo_export_20190314
Processing: Geo_export_20190322
Export task created for Geo_export_20190322
Processing: Geo_export_20190330
Export task created for Geo_export_20190330
Processing: Geo_export_20190407
Export task created for Geo_export_20190407
Processing: 